In [2]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

<ipython-input-3-dec148410aec>:3: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [4]:
import getpass
es_cloud_id = getpass.getpass("Enter Elastic Cloud ID: ")
api_id = getpass.getpass("Enter Elastic API ID: ")
api_key = getpass.getpass("Enter Elastic API KEY: ")

# Create the client instance
client = Elasticsearch(cloud_id=es_cloud_id, api_key=(api_id, api_key))
client.info()

Enter Elastic Cloud ID: ··········
Enter Elastic API ID: ··········
Enter Elastic API KEY: ··········


ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': '5089524370ab4e1396e3270ca581c541', 'cluster_uuid': 'LFC-6cmGSsqshnnawgItBg', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
es_model_id = 'intfloat__multilingual-e5-base'

In [6]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:

culture_info = pd.read_csv('/content/drive/MyDrive/빅데이터자연어처리프로젝트/data/merge_selected_data.csv', encoding='utf-8')
culture_info.loc[:1]

,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY,price,contents1,placeAddr
0,250002,토비아스 레베르거: 가끔이나마 내 마음대로 할 수 있는 곳은 나 자신 뿐이다,20180616.0,20251231.0,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/23/07/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관
1,319005,패트릭 블랑: 수직정원,20180616.0,20281231.0,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/25/04/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관


In [8]:
# ES date type 필드에 색인하기 위한 시작 종료일자 타입 변경
def parse_date(date_str):
    if pd.isna(date_str):
        return None
    try:
        return pd.to_datetime(str(int(date_str)).split('.')[0], format='%Y%m%d')
    except ValueError:
        return None

culture_info['startDate'] = culture_info['startDate'].apply(parse_date)
culture_info['endDate'] = culture_info['endDate'].apply(parse_date)

culture_info['startDate'] = culture_info['startDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
culture_info['endDate'] = culture_info['endDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

culture_info.loc[:1]

,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY,price,contents1,placeAddr
0,250002,토비아스 레베르거: 가끔이나마 내 마음대로 할 수 있는 곳은 나 자신 뿐이다,2018-06-16T00:00:00.000000Z,2025-12-31T00:00:00.000000Z,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/23/07/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관
1,319005,패트릭 블랑: 수직정원,2018-06-16T00:00:00.000000Z,2028-12-31T00:00:00.000000Z,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/25/04/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관


In [9]:
#공연 정보 생성 (유형, 지역, 장소, 공연명을 더해서 공연정보 생성)
culture_info['information'] = "[" + culture_info['realmName'] + "] " + culture_info['area'] +  " " + culture_info['sigungu'] + " : " + culture_info['place']  + " : " + culture_info['title']
culture_info.loc[:3]

,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY,price,contents1,placeAddr,information
0,250002,토비아스 레베르거: 가끔이나마 내 마음대로 할 수 있는 곳은 나 자신 뿐이다,2018-06-16T00:00:00.000000Z,2025-12-31T00:00:00.000000Z,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/23/07/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관,[전시] 부산 사하구 : 부산현대미술관 : 토비아스 레베르거: 가끔이나마 내 마음대...
1,319005,패트릭 블랑: 수직정원,2018-06-16T00:00:00.000000Z,2028-12-31T00:00:00.000000Z,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/25/04/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관,[전시] 부산 사하구 : 부산현대미술관 : 패트릭 블랑: 수직정원
2,267899,"홍영철의 유산, 부산영화를 담다",2018-10-05T00:00:00.000000Z,2030-12-31T00:00:00.000000Z,부산영화체험박물관,전시,부산,중구,http://www.culture.go.kr/upload/rdf/24/01/show...,129.033711,35.101740,"성인 10,000원 / 어린이, 청소년 7,000원",NaN,부산광역시 중구 대청로126번길 12 부산영화체험박물관,"[전시] 부산 중구 : 부산영화체험박물관 : 홍영철의 유산, 부산영화를 담다"
3,247148,[국립중앙도서관 상설 전시] 시간의 기록을 잇다,2018-10-29T00:00:00.000000Z,2025-12-31T00:00:00.000000Z,국립중앙도서관,전시,서울,서초구,http://www.culture.go.kr/upload/rdf/23/06/show...,127.002985,37.497618,무료,NaN,서울특별시 서초구 반포대로 201 국립중앙도서관,[전시] 서울 서초구 : 국립중앙도서관 : [국립중앙도서관 상설 전시] 시간의 기록...


In [13]:
# 공연타입, 지역 정보가 없는 데이터는 유효하지 않은데이터로 제외
culture_info.dropna(subset=['information'], inplace=True)

Rows with NaN in 'information' column removed.


,seq,title,startDate,endDate,place,realmName,area,sigungu,thumbnail,gpsX,gpsY,price,contents1,placeAddr,information
0,250002,토비아스 레베르거: 가끔이나마 내 마음대로 할 수 있는 곳은 나 자신 뿐이다,2018-06-16T00:00:00.000000Z,2025-12-31T00:00:00.000000Z,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/23/07/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관,[전시] 부산 사하구 : 부산현대미술관 : 토비아스 레베르거: 가끔이나마 내 마음대...
1,319005,패트릭 블랑: 수직정원,2018-06-16T00:00:00.000000Z,2028-12-31T00:00:00.000000Z,부산현대미술관,전시,부산,사하구,http://www.culture.go.kr/upload/rdf/25/04/show...,128.942742,35.109216,무료,NaN,부산광역시 사하구 낙동남로 1191 부산 현대미술관,[전시] 부산 사하구 : 부산현대미술관 : 패트릭 블랑: 수직정원
2,267899,"홍영철의 유산, 부산영화를 담다",2018-10-05T00:00:00.000000Z,2030-12-31T00:00:00.000000Z,부산영화체험박물관,전시,부산,중구,http://www.culture.go.kr/upload/rdf/24/01/show...,129.033711,35.101740,"성인 10,000원 / 어린이, 청소년 7,000원",NaN,부산광역시 중구 대청로126번길 12 부산영화체험박물관,"[전시] 부산 중구 : 부산영화체험박물관 : 홍영철의 유산, 부산영화를 담다"
3,247148,[국립중앙도서관 상설 전시] 시간의 기록을 잇다,2018-10-29T00:00:00.000000Z,2025-12-31T00:00:00.000000Z,국립중앙도서관,전시,서울,서초구,http://www.culture.go.kr/upload/rdf/23/06/show...,127.002985,37.497618,무료,NaN,서울특별시 서초구 반포대로 201 국립중앙도서관,[전시] 서울 서초구 : 국립중앙도서관 : [국립중앙도서관 상설 전시] 시간의 기록...
4,300600,밀양아리랑 전시관 상설전시,2019-03-14T00:00:00.000000Z,2025-12-31T00:00:00.000000Z,밀양아리랑아트센터,전시,경남,밀양시,http://www.culture.go.kr/upload/rdf/24/11/show...,128.758410,35.504412,무료,NaN,경상남도 밀양시 밀양대공원로 112 밀양아리랑아트센터,[전시] 경남 밀양시 : 밀양아리랑아트센터 : 밀양아리랑 전시관 상설전시


In [19]:
len(culture_info)

1435

In [15]:
# Define the mapping and settings
mapping = {
    "properties": {
        "name": {"type": "text"},
        "type": {"type": "keyword"},
        "area": {"type": "keyword"},
        "place": {"type": "keyword"},
        "address": {"type": "text"},
        "price": {"type": "text"},
        "startDate": {"type": "date"},
        "endDate": {"type": "date"},
        "location" : {"type": "geo_point"},
        "information": {"type": "text"},
        "information_embedding.predicted_value": {
            "type": "dense_vector",
            "dims": 768,
            "index": "true",
            "similarity": "cosine"
        }
    }
}

settings = {
    "index": {
        "number_of_replicas": "1",
        "number_of_shards": "1",
        "default_pipeline": "pipeline",
    }
}

# Create the index (deleting any existing index)
client.indices.delete(index="culture", ignore_unavailable=True)
client.indices.create(index="culture", mappings=mapping, settings=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'culture'})

In [18]:
indexed_results = []

for index, row in culture_info.iterrows():
    doc_source = {
        "name": row['title'] if pd.notna(row['title']) else None,
        "type": row['realmName'] if pd.notna(row['realmName']) else None,
        "area": row['area'] if pd.notna(row['area']) else None,
        "place": row['place'] if pd.notna(row['place']) else None,
        "address": row['placeAddr'] if pd.notna(row['placeAddr']) else None,
        "price": row['price'] if pd.notna(row['price']) else None,
        "startDate": row['startDate'] if pd.notna(row['startDate']) else None,
        "endDate": row['endDate'] if pd.notna(row['endDate']) else None,
        "information": row['information'] if pd.notna(row['information']) else None
    }

    # 로케이션 정보가 nan이면 색인시점에 에러발생. 따라서 location 값이 nan이면 null로 저장하기 위해 체크
    if pd.notna(row['gpsY']) and pd.notna(row['gpsX']):
         doc_source["location"] = {
            "lat": row['gpsY'],
            "lon": row['gpsX']
        }
    else:
        doc_source["location"] = None # 로케이션 정보가 유효하지 않은 데이터는 로케이션 값을 null로 저장


    try:
        response = client.index(index="culture", document=doc_source, id=str(index))
        indexed_results.append({'index': index, 'status': 'success', 'response': response})
        if index % 100 == 0:
            print(f"Successfully indexed document with index {index}")
    except Exception as e:
        indexed_results.append({'index': index, 'status': 'failed', 'error': str(e)})
        print(f"Failed to index document with index {index}: {e}")


Successfully indexed document with index 0
Successfully indexed document with index 100
Successfully indexed document with index 200
Successfully indexed document with index 300
Successfully indexed document with index 500
Successfully indexed document with index 600
Successfully indexed document with index 700
Successfully indexed document with index 800
Successfully indexed document with index 900
Successfully indexed document with index 1000
Successfully indexed document with index 1100
Successfully indexed document with index 1200
Successfully indexed document with index 1300
Successfully indexed document with index 1400
Successfully indexed document with index 1500


In [21]:
len(culture_info)

1435